<a href="https://colab.research.google.com/github/arpitpandey2/basicneuralnetwork/blob/main/basicneuralnetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [280]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [281]:
# Create our model
class Model(nn.Module):
  def __init__(self,in_features=4,h1=8,h2=9,out_features=3):
    super().__init__()
    self.fc1= nn.Linear(in_features,h1)
    self.fc2= nn.Linear(h1,h2)
    self.out= nn.Linear(h2,out_features)

  def forward(self,x):
    x= f.relu(self.fc1(x))
    x= f.relu(self.fc2(x))
    x= self.out(x)

    return x

In [282]:
# Feed the data
torch.manual_seed(30)
mode1= Model()

In [ ]:
# Load the data
url= 'https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv'
my_df= pd.read_csv(url)
my_df

In [ ]:
# Replace objects to numbers, so to train easily
my_df['species']=my_df['species'].replace('setosa',0.0)
my_df['species']=my_df['species'].replace('versicolor',1.0)
my_df['species']=my_df['species'].replace('virginica',2.0)
my_df['species'] = my_df['species'].astype(int)
my_df

In [285]:
# Prepare data for splitting
X=my_df.drop('species',axis=1)
Y=my_df['species']
x=X.values
y=Y.values

In [ ]:
x
y

In [287]:
# Import train_test_split
from sklearn.model_selection import train_test_split

In [288]:
# Split data into training and testing sets
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2, random_state=40)

In [290]:
# Convert data to PyTorch tensors
x_train=torch.FloatTensor(x_train)
x_test=torch.FloatTensor(x_test)

y_train=torch.LongTensor(y_train)
y_test=torch.LongTensor(y_test)

In [291]:
# Define criterion and optimizer
criterion= nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(mode1.parameters(), lr=0.01)

In [ ]:
# Train the model
epochs=100
losses=[]
for i in range(epochs):
  y_pred= mode1.forward(x_train)
  loss= criterion(y_pred,y_train)
  losses.append(loss.detach().numpy())

  if i%10==0:
    print(f"Epoch: {i} and loss {loss}")

  # Backprop and optimize
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

In [ ]:
# Plot the loss
plt.plot(range(epochs),losses)
plt.ylabel('Loss')
plt.xlabel('Epoch')

In [ ]:
with torch.no_grad():
  y_eval= mode1.forward(x_test)
  loss= criterion(y_eval,y_test)
loss

In [ ]:
correct= 0
with torch.no_grad():
  for i, data in enumerate(x_test):
    y_val= mode1.forward(data)
    print(f'{i+1:2}. {str(y_val):38} {y_test[i]}')
    if y_val.argmax().item()== y_test[i]:
      correct+=1

print(f'total corrects are {correct}')

New data and check our neural network

In [298]:
new_iris= torch.tensor([4.7,3.2,1.3,0.3])

In [ ]:
with torch.no_grad():
  print(mode1(new_iris))
  print(mode1(new_iris).argmax().item())


Save and check our model

In [303]:

torch.save(mode1.state_dict(), 'my_iris_model.pth')


In [305]:
# Load the saved model
loaded_model = Model()
loaded_model.load_state_dict(torch.load('my_iris_model.pth'))

# Test the loaded model with new data
new_iris_test = torch.tensor([4.7,3.2,1.3,0.3])
with torch.no_grad():
  print(loaded_model(new_iris_test))
  print(loaded_model(new_iris_test).argmax().item())

tensor([11.2693,  5.2038, -9.6135])
0
